# Проект по SQL

### План

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Цель исследования

Проанализировать базу данных для выпуска приложения с наиболее популярными книгами, издательствами, авторами

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


###### Найдем какое количество книг вышло после 1 января 2000 года;

In [6]:
query = '''SELECT COUNT(book_id) 
FROM books 
WHERE publication_date>'2000-01-01' 
LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Вывод: за период с 01/01/2000 было выпущено 819 книг

###### Посчитаем количество обзоров и среднюю оценку;

In [12]:
query = '''SELECT bo.book_id,
bo.title,
AVG(ra.rating) AS mean_rating,
COUNT(DISTINCT re.review_id) AS review_quantity
FROM books AS bo
LEFT OUTER JOIN ratings AS ra ON bo.book_id=ra.book_id
LEFT OUTER JOIN reviews AS re ON bo.book_id=re.book_id
GROUP BY bo.book_id
ORDER BY COUNT(DISTINCT re.review_id) DESC'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,mean_rating,review_quantity
0,948,Twilight (Twilight #1),3.662500,7
1,963,Water for Elephants,3.977273,6
2,734,The Glass Castle,4.206897,6
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
4,695,The Curious Incident of the Dog in the Night-Time,4.081081,6
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0
996,808,The Natural Way to Draw,3.000000,0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
998,221,Essential Tales and Poems,4.000000,0


Всего по выгрузке получилось 1000 книг. Каждому ID книги соответсвует количество обзоров и средний рейтинг. Количество обзоров вариируется в пределах от 0 до 7. Максимальное количество 7 по книге с ID 948. Средние рейтинги вариируеются от 1,5 до 5. 5-максимальный рейтинг.

###### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [8]:
query = '''SELECT pu.publisher,
COUNT(bo.book_id) AS quantity
FROM books AS bo
JOIN publishers AS pu ON bo.publisher_id=pu.publisher_id
WHERE bo.num_pages>50
GROUP BY pu.publisher
ORDER BY COUNT(bo.book_id) DESC'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,quantity
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


По примененным фильтрам получилось 334 издательства, которые выпустили книги, тольщиной более 50 страниц. Первая 5-ка по количеству книг: Penguin Books-42, Vintage-31, Grand Central Publishing-25, Penguin Classics-24, Ballantine Books-19. Последняя 5-ка: Turtleback - 1, Atheneum Books for Young Readers: Richard Jack - 1, Penguin Signet- 1, Victor Gollancz-1, Harvard Business Review Press - 1.

###### Найдем автора с самой высокой средней оценкой книг (только книги с 50 и более оценками)

In [9]:
query = '''SELECT au.author, 
AVG(ra.rating)
FROM authors as au
LEFT JOIN books as bo ON au.author_id = bo.author_id
LEFT JOIN ratings as ra ON bo.book_id= ra.book_id
WHERE bo.book_id in (SELECT book_id
                    from ratings
                    group by book_id
                    having count(rating)>=50
                    order by count(rating) desc)
GROUP BY au.author
ORDER BY AVG(ra.rating) DESC
LIMIT 1
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой, по книгам которого более 50 оценок J.K. Rowling/Mary GrandPré - 4.288462

###### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [10]:
query = '''

SELECT AVG(r.quantity)
FROM (SELECT username,
    COUNT(review_id) AS quantity
    FROM reviews
    WHERE username in (SELECT username
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating)>48)
    GROUP BY username) AS r
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24

In [11]:
#КОД РЕВЬЮЕРА
query = '''SELECT COUNT(*) FROM reviews'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


Итоговый вывод:
Было изучено 5 датасетов, выполнено 5 заданий со следующим результатом:

1. за период с 01/01/2000 было выпущено 819 книг
2. Всего по выгрузке получилось 1000 книг. Каждому ID книги соответсвует количество обзоров и средний рейтинг. Количество обзоров вариируется в пределах от 0 до 7. Максимальное количество 7 по книге с ID 948. Средние рейтинги вариируеются от 1,5 до 5. 5-максимальный рейтинг.
3. По примененным фильтрам получилось 334 издательства, которые выпустили книги, тольщиной более 50 страниц. Первая 5-ка по количеству книг: Penguin Books-42, Vintage-31, Grand Central Publishing-25, Penguin Classics-24, Ballantine Books-19. Последняя 5-ка: Turtleback - 1, Atheneum Books for Young Readers: Richard Jack - 1, Penguin Signet- 1, Victor Gollancz-1, Harvard Business Review Press - 1.
4. Автор с самой высокой средней оценкой, по книгам которого более 50 оценок J.K. Rowling/Mary GrandPré - 4.288462
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24